In [1]:
#Task 2 Part 2
#Contributors : Eunice Ofori Addo, Ajinkya Borle
#Based on binary search, at least in this case we know what the upper limit of our data is :)
import numpy as np
import torch
from RBM_helper import RBM

import Rydberg_energy_calculator

training_data = torch.from_numpy(np.loadtxt("Rydberg_data.txt"))


In [2]:
n_vis = training_data.shape[1]
#twice the neurons obtained from the las
n_hin =  2 * 50 #placeholder please replace with actual value

rbm = RBM(n_vis, n_hin)

In [3]:
#Let us try to figure out how little data we can use
#Define all parameters for the RBM that won't change within the loop
epochs = 500 #
num_samples = 2000 #increased the number of samples for better accuracy of approximating the expectation value
k = 10 #Having too huge of a k is painful, and some studies have shown k=1 to also work well

#Now we define the variables that are specific to our 'intelligent guesswork', inspired by binary search
R = 20000 - 1 #This variable is how high can we go for the dataset (0 based index that's why -1)
L = 0 #This variable is how low can we go for n_hin (0 based index)


temp_datanum = -1 #This variable holds the temporary candidate for final size

exact_energy = -4.1203519096
print("Exact energy: ",exact_energy)

while L <= R:
    
    
    m = int((L + R)/2)
    data_subset = training_data[0:m+1]
    
    print("Upper limit is (0 based):",R)
    print("Lower limit is (0 based):",L)
    print("Current dataset size is:",m + 1)
    rbm = RBM(n_vis, n_hin) #Create an rbm object here
    
    for e in range(1, epochs+1):
        # do one epoch of training
        rbm.train(data_subset)   

        # now generate samples and calculate the energy
        if e % 100 == 0:
            print("\nEpoch: ", e)
            print("Sampling...")

            init_state = torch.zeros(num_samples, n_vis)
            RBM_samples = rbm.draw_samples(k, init_state)

            print("Done sampling. Calculating energy...") 
            
            energies = Rydberg_energy_calculator.energy(RBM_samples, rbm.wavefunction) 
            print("Energy from RBM samples: ", energies.item())
    #After total epochs done
    print("---")
    if abs(energies.item() - exact_energy) < 0.0001:
        print("Criteria reached, let's see if we can get a better estimate of size of data")
        temp_datanum = m + 1
        R = m - 1
    else:
        L = m + 1
        print("Criteria not reached")
if temp_datanum != -1:
    print("Least data size we need:", temp_datanum)
else:
    print("Trying increasing epoch numbers, k and/or num_samples")

Exact energy:  -4.1203519096
Upper limit is (0 based): 19999
Lower limit is (0 based): 0
Current dataset size is: 10000

Epoch:  100
Sampling...
Done sampling. Calculating energy...


..\aten\src\ATen\native\BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.


Energy from RBM samples:  -4.1190811701814365

Epoch:  200
Sampling...
Done sampling. Calculating energy...
Energy from RBM samples:  -4.11892156680444

Epoch:  300
Sampling...
Done sampling. Calculating energy...
Energy from RBM samples:  -4.11881837451076

Epoch:  400
Sampling...
Done sampling. Calculating energy...
Energy from RBM samples:  -4.1194850676463215

Epoch:  500
Sampling...
Done sampling. Calculating energy...
Energy from RBM samples:  -4.119095257685325
---
Criteria not reached
Upper limit is (0 based): 19999
Lower limit is (0 based): 10000
Current dataset size is: 15000

Epoch:  100
Sampling...
Done sampling. Calculating energy...
Energy from RBM samples:  -4.119324245745185


KeyboardInterrupt: 